In [2]:
import string
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import sqlite3
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem import PorterStemmer
# nltk.download('stopwords')

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics

In [3]:
conn = sqlite3.connect('../datafiles/amazon_reviews.sqlite')
data = pd.read_sql_query(""" SELECT * FROM Reviews WHERE Score!=3""",conn)

### Data Cleansing

In [4]:
def scr(s):
    if(s>3):
        return 'positive'
    else:
        return 'negative'

In [5]:
data['Score']

0         5
1         1
2         4
3         2
4         5
         ..
525809    5
525810    2
525811    5
525812    5
525813    5
Name: Score, Length: 525814, dtype: int64

In [6]:
data['Score'] = data['Score'].apply(scr)

In [7]:
data.columns

Index(['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator',
       'HelpfulnessDenominator', 'Score', 'Time', 'Summary', 'Text'],
      dtype='object')

In [8]:
cus_data = data.drop_duplicates(subset={'UserId','ProfileName', 'Time', 'Text'},keep='first')
cus_data = cus_data[cus_data['HelpfulnessNumerator']<=cus_data['HelpfulnessDenominator']]
cus_data.shape

(364171, 10)

In [9]:
cus_data['Score'].value_counts()

positive    307061
negative     57110
Name: Score, dtype: int64

In [10]:
cus_data.iloc[1500]['Text']

'Aboulutely love Popchips!I first tried these healthy chips at a marathon i did in California. I like this variety pack because i got to try alot of the flavors ive never had.'

>>>### Text Preprosessing

- Begin by removing the html tags
- Remove any punctuations or limited set of special characters like , or . or # etc.
- Check if the word is made up of english letters and is not alpha-numeric
- Check to see if the length of the word is greater than 2 (as it was researched that there is no adjective in 2-letters)
- Convert the word to lowercase
- Remove Stopwords
- Finally Snowball Stemming the word (it was obsereved to be better than Porter Stemming)

After which we collect the words used to describe positive and negative reviews

>- ## cleaning html tags

In [11]:
def cln_html(sen):
    clnd = re.sub(r'<.*?>',r' ',sen)
    return clnd
def cln_punc(sen):
    clnd = re.sub(r'[?|!|\'|"|#]',r'',sen)
    clnd = re.sub(r'[.|,|)|(|\|/]',r' ',clnd)
    return clnd    
stop = set(stopwords.words('english'))
sno = nltk.stem.SnowballStemmer('english') 

In [12]:
i=0
str1=' '
final_string=[]
all_positive_words=[]
all_negative_words=[]
s=''
for sent in cus_data['Text'].values:
    filtered_sentence=[]
    sent=cln_html(sent)
    for w in sent.split():
        for cleaned_words in cln_punc(w).split():
            if((cleaned_words.isalpha()) & (len(cleaned_words)>2)):    
                if(cleaned_words.lower() not in stop):
                    s=(sno.stem(cleaned_words.lower())).encode('utf8')
                    filtered_sentence.append(s)
                    if (cus_data['Score'].values)[i] == 'positive': 
                        all_positive_words.append(s)
                    if(cus_data['Score'].values)[i] == 'negative':
                        all_negative_words.append(s)
                else:
                    continue
            else:
                continue 
    str1 = b" ".join(filtered_sentence)
    final_string.append(str1)
    i+=1

In [13]:
cus_data['CleanedText']=final_string

In [14]:
conn = sqlite3.connect('cus_data.sqlite')
c=conn.cursor()
conn.text_factory = str
cus_data.to_sql('Reviews', conn, schema=None, if_exists='replace', index=True, index_label=None, chunksize=None, dtype=None)